In [1]:
from sql_metadata import Parser
import pandas as pd

In [45]:
query = """
SELECT "T0"."C0" "ViCO_LS_ID",
       "T0"."C1" "lscpostord",
       "T0"."C2" "VICOrd_UpdateDate",
       "T0"."C3" "VICOrdLin_LineId",
       "T0"."C4" "VICOrdLin_VolumePlan",
       "T0"."C5" "VICOrdLin_VolumeTotal",
       "T0"."C6" "VICOrdLin_UpdateDate",
       "T0"."C7" "VICOrdLin_QtyShip",
       "T0"."C8" "VICOrd_BuCodeRcv",
       "T0"."C9" "ViCO_CusOrd_ID",
       "T0"."C10" "Count_ViCO_LS_ID_",
       "T0"."C11" "Count_ViCO_CusOrd_ID_",
       "T0"."C12" "Count_VICOrdLin_LineId_",
       "T0"."C5" "Total_VICOrdLin_VolumeTotal_",
       "T0"."C7" "Total_VICOrdLin_QtyShip_",
       "T0"."C7" "VICOrdLin_QtyPick"
FROM
  (SELECT CASE
              WHEN to_char("R08T1"."OPPDATE", 'DDMMYYYY') IS NULL
                   OR trim(BOTH
                           FROM "R08T1"."ROUTENO") IS NULL
                   OR trim(BOTH
                           FROM "R08T1"."TRIPNO") IS NULL THEN NULL
              ELSE to_char("R08T1"."OPPDATE", 'DDMMYYYY') || '-' || trim(BOTH
                                                                         FROM "R08T1"."ROUTENO") || '-' || trim(BOTH
                                                                                                                FROM "R08T1"."TRIPNO")
          END "C0",
          CASE
              WHEN "O04T90"."RCVBUCOD" IN ('857',
                                           '801') THEN 'POST'
              ELSE 'LSC'
          END "C1",
          TRUNC(cast("O04T14"."STATDATE" AS TIMESTAMP(9))) "C2",
          trim(BOTH
               FROM to_char("O08T15"."ORDLINE")) "C3",
          sum("O08T15"."CALCVOL"/1000) "C4",
          sum("O08T15"."REALVOL"/1000) "C5",
          "O08T15"."STATDATE" "C6",
          sum("O08T15"."DELQUANT") "C7",
          "O04T90"."RCVBUCOD" "C8",
          "O04T14"."EORDERID" "C9",
          min(CASE
                  WHEN to_char("R08T1"."OPPDATE", 'DDMMYYYY') IS NULL
                       OR trim(BOTH
                               FROM "R08T1"."ROUTENO") IS NULL
                       OR trim(BOTH
                               FROM "R08T1"."TRIPNO") IS NULL THEN NULL
                  ELSE to_char("R08T1"."OPPDATE", 'DDMMYYYY') || '-' || trim(BOTH
                                                                             FROM "R08T1"."ROUTENO") || '-' || trim(BOTH
                                                                                                                    FROM "R08T1"."TRIPNO")
              END) "C10",
          min("O04T14"."EORDERID") "C11",
          min(trim(BOTH
                   FROM to_char("O08T15"."ORDLINE"))) "C12"
   FROM "R08T1" "R08T1",
        "O04T90" "O04T90",
        "O04T14",
        "O08T15"
   WHERE CASE
             WHEN ("O08T15"."LINESTAT"=1) THEN 10
             WHEN ("O08T15"."LINESTAT"=22) THEN 20
             WHEN ("O08T15"."LINESTAT"=2) THEN 30
             WHEN ("O08T15"."LINESTAT"=7) THEN 50
             WHEN ("O08T15"."LINESTAT"=0) THEN 99
             ELSE NULL
         END ='50'
     AND TRUNC(cast("O04T14"."STATDATE" AS TIMESTAMP(9))) BETWEEN TRUNC(cast(:PQ1 AS TIMESTAMP(9))) AND TRUNC(cast(:PQ2 AS TIMESTAMP(9)))
     AND CASE
             WHEN ("O04T14"."ORDSTAT"=1) THEN 10
             WHEN ("O04T14"."ORDSTAT"=22) THEN 20
             WHEN ("O04T14"."ORDSTAT"=2) THEN 30
             WHEN ("O04T14"."ORDSTAT"=7) THEN 40
             WHEN ("O04T14"."ORDSTAT"=9) THEN 50
             WHEN ("O04T14"."ORDSTAT"=0) THEN 99
             ELSE NULL
         END ='50'
     AND "O08T15"."SHORTR08"="R08T1"."SHORTR08"
     AND "O08T15"."SHORTO04"="O04T14"."SHORTO04"
     AND "O04T14"."SHORTO04"="O04T90"."SHORTO04"
   GROUP BY CASE
                WHEN to_char("R08T1"."OPPDATE", 'DDMMYYYY') IS NULL
                     OR trim(BOTH
                             FROM "R08T1"."ROUTENO") IS NULL
                     OR trim(BOTH
                             FROM "R08T1"."TRIPNO") IS NULL THEN NULL
                ELSE to_char("R08T1"."OPPDATE", 'DDMMYYYY') || '-' || trim(BOTH
                                                                           FROM "R08T1"."ROUTENO") || '-' || trim(BOTH
                                                                                                                  FROM "R08T1"."TRIPNO")
            END,
            CASE
                WHEN "O04T90"."RCVBUCOD" IN ('857',
                                             '801') THEN 'POST'
                ELSE 'LSC'
            END,
            TRUNC(cast("O04T14"."STATDATE" AS TIMESTAMP(9))),
            trim(BOTH
                 FROM to_char("O08T15"."ORDLINE")),
            "O08T15"."STATDATE",
            "O04T90"."RCVBUCOD",
            "O04T14"."EORDERID") "T0"
"""

In [52]:

import re

pattern = r'"[^"]*"\.'

# Ersetzen
result = re.sub(pattern, '', query)

In [53]:
# print(result)

In [54]:
parser = Parser(query)

In [55]:
parser.tables

['R08T1.ROUTENO',
 'BOTH',
 'R08T1.TRIPNO',
 'R08T1.OPPDATE',
 'CASE',
 'TRUNC',
 'cast',
 'O04T14.STATDATE',
 'trim',
 'to_char',
 'O08T15.ORDLINE',
 'sum',
 'O08T15.CALCVOL',
 'O08T15.REALVOL',
 'O08T15.STATDATE',
 'O08T15.DELQUANT',
 'O04T90.RCVBUCOD',
 'O04T14.EORDERID',
 'min',
 'R08T1',
 'O04T90',
 'O04T14',
 'O08T15']

In [56]:
parser.columns

ValueError: 'C0' is not in list

In [57]:
parser.columns_aliases

ValueError: 'C0' is not in list

In [58]:
parser.sqlparse_tokens

[<Newline ' ' at 0x23A8340FFA0>,
 <DML 'SELECT' at 0x23A8340FA00>,
 <Whitespace ' ' at 0x23A8340F580>,
 <IdentifierList '`T0`.`...' at 0x23A836B8FD0>,
 <Newline ' ' at 0x23A83665CC0>,
 <Keyword 'FROM' at 0x23A83665BA0>,
 <Newline ' ' at 0x23A83666200>,
 <Whitespace ' ' at 0x23A83666020>,
 <Whitespace ' ' at 0x23A83666380>,
 <Identifier '(SELEC...' at 0x23A836B8CD0>,
 <Newline ' ' at 0x23AFFD66E00>]